Library

In [1]:
!pip install transformers torch scikit-learn sentencepiece sacremoses

In [2]:
!pip install openpyxl

Code - 1 - webscraping

In [3]:
import pandas as pd
import trafilatura
import time

# Caminho para o arquivo CSV
file_path = './database/finance_news.csv'

# Carrega o arquivo CSV em um DataFrame
df = pd.read_csv(file_path)

def extrair_texto_principal(url):
    try:
        # Baixar e extrair o texto principal da URL
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            texto_principal = trafilatura.extract(downloaded, output_format='txt')
            if texto_principal:
                return texto_principal
            else:
                return "Não foi possível encontrar o conteúdo principal."
        else:
            return "Erro ao baixar o conteúdo da página."
    except Exception as e:
        return f"Erro ao extrair texto de {url}: {e}"

# Inicializa a lista de resumos
resumos_lista = []

# Extrai a coluna 'links' e 'title' e converte para listas
urls = df['link'].tolist()
titles = df['title'].tolist()

# Lista para armazenar os dados da nova tabela com notícias completas
data = []

for title, url in zip(titles, urls):
    texto_completo = extrair_texto_principal(url)  # Extrai o texto principal da URL
    data.append({'noticias': texto_completo})
    
    # A lista vai armazenar o texto resumido
    resumos_lista.append(f'{title}: {texto_completo[:200]}...')  # Exemplo de resumo básico
    
    time.sleep(1)  # Espera 1 segundo entre as requisições

# Cria um novo DataFrame com os dados das notícias completas
df_final = pd.DataFrame(data, columns=['noticias'])

# Salvar a tabela em um arquivo Excel
file_output_path = './database/produto.xlsx'
df_final.to_excel(file_output_path, index=False)

# Exibe a lista de resumos
print("Lista de resumos das notícias:")
for resumo in resumos_lista:
    print(resumo)

# Exibe uma mensagem de sucesso
print(f"Arquivo salvo com sucesso em {file_output_path}")

Lista de resumos das notícias:
Abertura econômica é caminho para desenvolvimento: Erro ao baixar o conteúdo da página....
[Removed]: Erro ao baixar o conteúdo da página....
[Removed]: Erro ao baixar o conteúdo da página....
[Removed]: Erro ao baixar o conteúdo da página....
Mais política industrial verde: Diante do movimento global rumo a uma economia de baixo carbono, são necessárias mais políticas industriais verdes que deem sinais claros de que os tomadores de decisão estão no caminho correto da tra...
Alguém está ouvindo o que diz a Ciência?: O mundo está prestes a ultrapassar a sétima das nove fronteiras planetárias definidas por cientistas como limites que a humanidade não deveria superar se quiser ficar em uma zona segura. São os sistem...
Alto Escalão: VR troca de comando: Alto Escalão: VR troca de comando
Veja essa e outras movimentações corporativas da semana
Simone Marques, até então sócia na VR Investimentos, passa a CEO do grupo, no lugar de Claudio Szajman, que se...
Tecn

Code - 2 - sentiments

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import MarianMTModel, MarianTokenizer
from torch.nn import functional as F

# Carregar o modelo e tokenizer do FinBERT
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Carregar o modelo de tradução multilíngue MarianMT (pt -> en)
modelo_traducao = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-mul-en')
tokenizer_traducao = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-mul-en')

# Função para traduzir de português para inglês
def traduzir(texto_pt):
    tokens = tokenizer_traducao(texto_pt, return_tensors="pt", padding=True)
    traducao = modelo_traducao.generate(**tokens)
    texto_traduzido = tokenizer_traducao.decode(traducao[0], skip_special_tokens=True)
    return texto_traduzido

# Função para realizar análise de sentimento diretamente com o FinBERT e obter as probabilidades
def analisar_sentimento(texto):
    inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    # Aplicar softmax para obter as probabilidades
    probabilidades = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    
    # O FinBERT foi treinado para 3 classes: 0 (negativo), 1 (neutro), 2 (positivo)
    sentimentos = {
        "positive": round(probabilidades[2] * 100, 2),
        "neutral": round(probabilidades[1] * 100, 2),
        "negative": round(probabilidades[0] * 100, 2)
    }
    return sentimentos

# Listas de resumos das notícias (você já tem isso no código anterior)
# A lista `resumos_lista` deve conter exatamente o número de resumos esperado
resumos_lista = [
    # Adicione os resumos das notícias aqui, conforme gerado no código anterior.
]

# Verifique se o número de resumos coincide com o número de linhas no arquivo Excel
file_path = './database/produto.xlsx'
df_produto = pd.read_excel(file_path)

if len(resumos_lista) != len(df_produto):
    raise ValueError("O número de resumos não corresponde ao número de notícias no arquivo Excel.")

# Lista para armazenar o maior sentimento de cada notícia
maiores_sentimentos = []

# Loop para analisar cada resumo
for resumo in resumos_lista:
    # Traduzir o resumo para inglês
    resumo_traduzido = traduzir(resumo)

    # Realizar a análise de sentimento no resumo traduzido
    sentimentos = analisar_sentimento(resumo_traduzido)

    # Determinar o sentimento com a maior porcentagem
    sentimento_maior = max(sentimentos, key=sentimentos.get)
    porcentagem_maior = sentimentos[sentimento_maior]

    # Adicionar o sentimento e a porcentagem correspondente na lista
    maiores_sentimentos.append(f"{sentimento_maior} ({porcentagem_maior}%)")

# Adicionar a nova coluna "sentimentos" ao DataFrame com os maiores sentimentos
df_produto['sentimentos'] = maiores_sentimentos

# Salvar o DataFrame atualizado de volta no arquivo Excel
df_produto.to_excel(file_path, index=False)

# Exibir uma mensagem de sucesso
print(f"Arquivo 'produto.xlsx' atualizado com sucesso!")


/workspaces/SynaptaInvest/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ValueError: O número de resumos não corresponde ao número de notícias no arquivo Excel.